In [1]:
import pandas as pd
import re

In [2]:
uniref = pd.read_csv("./data/2021-03-31.TettAJ_2016.gene_families.txt", sep="\t", index_col=0)

In [10]:
uniref.head()

,SK_CT101OSL_t1M14,SK_CT101RCR_t1M14,SK_CT102OSL_t1M14,SK_CT102RCL_t1M14,SK_CT102RCR_t1M14,SK_CT104OSL_t1M14,SK_CT104OSR_t1M14,SK_CT104RCL_t1M14,SK_CT104RCR_t1M14,SK_CT105OSL_t1M14,...,Steph_b2_64,Steph_b2_65,Steph_b2_66,Steph_b2_67,Steph_b2_68,Steph_b2_70,Steph_b2_71,Steph_b2_72,Steph_b2_74,Steph_b2_9
UNMAPPED,0.336727,0.859273,0.374585,2.179490e-01,0.291308,0.312706,0.266681,0.826913,0.822014,0.686935,...,0.578803,0.420364,0.186414,0.573109,0.834071,0.18572,0.817462,0.575684,0.436962,0.327175
UniRef90_C4PUD2,0.003075,0.000060,0.000000,5.410450e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
UniRef90_C4PUD0,0.002848,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
UniRef90_C4PUC9,0.002700,0.000000,0.000000,2.822040e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
UniRef90_C4PUC8,0.002671,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [4]:
urid_list = uniref.index.tolist()[2:]

In [5]:
id_list = [re.search(r'UniRef90_(\S+)', id_str).group(1) for id_str in urid_list]

In [6]:
extracted_ids = [re.search(r'uniref_id(\d+)', id_str).group(1) for id_str in id_list if re.search(r'uniref_id(\d+)', id_str)]


In [7]:
id_list_100000 = id_list[:100000]

In [21]:
with open('extracted_ids.txt', 'w') as f:
    for id in id_list:
        f.write(f"{id}\n")

In [3]:
uniref.shape

(558892, 89)

In [8]:
with open('extracted_ids_100000.txt', 'w') as f:
    for id in id_list_100000:
        f.write(f"{id}\n")

从uniprot中检索前100000个unirefid后，获取其关联的kegg id

In [1]:
import pandas as pd
keggid = pd.read_csv("./idmapping_100000.tsv",sep="\t")


In [3]:
keggid.head()

,From,Entry,KEGG
0,A0A009E9G3,A0A009E9G3,NaN
1,A0A009EC87,A0A009EC87,NaN
2,A0A009ECP6,A0A009ECP6,NaN
3,A0A009EJT5,A0A009EJT5,NaN
4,A0A009EM74,A0A009EM74,NaN


In [13]:
keggid = keggid[~keggid['KEGG'].isna()][['From','KEGG']] # 去掉没有对应KEGG号的行
keggid['KEGG'] = keggid['KEGG'].str.rstrip(';') # 去掉末尾的分号;
keggid = keggid.reset_index(drop=True)
keggid.head()

,From,KEGG
0,A0A023UEV9,shh:ShL2_00055
1,A0A023UGA4,shh:ShL2_00101
2,A0A028ZLY2,shh:ShL2_00073
3,A0A031FNR5,moo:BWL13_02343
4,A0A060AB44,vg:19685786


网页爬取，获取kegg中的结果

In [ ]:
import pandas as pd
import requests

def webtext2df(x):
    import pandas as pd
    rows = x.split('\n')
    data = [row.split('\t') for row in rows]
    df = pd.DataFrame(data, columns=['org:gene','ko'])
    df.drop(df.index[-1],inplace=True)
    return df

keggid = pd.read_csv("./idmapping_100000.tsv",sep="\t")

keggid = keggid[~keggid['KEGG'].isna()][['From','KEGG']] # 去掉没有对应KEGG号的行
keggid['KEGG'] = keggid['KEGG'].str.rstrip(';') # 去掉末尾的分号;
keggid = keggid.reset_index(drop=True)

keggid_list = '+'.join(keggid['KEGG'].head(100))

# url
url = f'https://rest.kegg.jp/link/ko/{keggid_list}'

# 发送 GET 请求 并获取响应
response = requests.get(url)

if response.status_code == 200:
    gene2ko = response.text
    print("请求成功！")
else :
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

gene2ko = webtext2df(gene2ko)

In [49]:
keggid['KEGG'][0:3]
'+'.join(keggid['KEGG'][0:3])

'shh:ShL2_00055+shh:ShL2_00101+shh:ShL2_00073'

In [50]:
import requests
import time

# 将 keggid_list 按 100 行分块
chunk_size = 100

# 初始化存储所有请求结果的变量
all_results = []

# 循环每个分块进行请求
for i in range(0, len(keggid['KEGG']), chunk_size):
    # 获取当前的 100 行数据
    current_chunk = keggid['KEGG'][i:i + chunk_size]
    
    # 将列表转换为以加号（+）连接的字符串，适合 URL 请求格式
    chunk_str = '+'.join(current_chunk)
    
    # 构建请求 URL
    url = f'https://rest.kegg.jp/link/ko/{chunk_str}'
    
    try:
        # 发送 GET 请求
        response = requests.get(url)
        
        if response.status_code == 200:
            print(f"请求成功！ 第 {i//chunk_size + 1} 批次")
            # 将请求的文本结果添加到总结果中
            all_results.append(response.text)
        else:
            print(f"请求失败！ HTTP 状态码: {response.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"请求发生错误: {e}")
    
    # 每次请求后睡眠2秒
    time.sleep(2)

# 将所有结果汇总为一个字符串
final_result = ''.join(all_results)

gene2ko = webtext2df(final_result)

请求成功！ 第 1 批次
请求成功！ 第 2 批次
请求成功！ 第 3 批次
请求成功！ 第 4 批次
请求成功！ 第 5 批次
请求成功！ 第 6 批次
请求成功！ 第 7 批次
请求成功！ 第 8 批次
请求成功！ 第 9 批次
请求成功！ 第 10 批次
请求成功！ 第 11 批次
请求成功！ 第 12 批次
请求成功！ 第 13 批次
请求成功！ 第 14 批次
请求成功！ 第 15 批次
请求成功！ 第 16 批次
请求成功！ 第 17 批次
请求成功！ 第 18 批次
请求成功！ 第 19 批次
请求成功！ 第 20 批次
请求成功！ 第 21 批次
请求成功！ 第 22 批次
请求成功！ 第 23 批次
请求成功！ 第 24 批次
请求成功！ 第 25 批次
请求成功！ 第 26 批次
请求成功！ 第 27 批次
请求成功！ 第 28 批次
请求成功！ 第 29 批次
请求成功！ 第 30 批次
请求成功！ 第 31 批次
请求成功！ 第 32 批次
请求成功！ 第 33 批次
请求成功！ 第 34 批次
请求成功！ 第 35 批次
请求成功！ 第 36 批次
请求成功！ 第 37 批次
请求成功！ 第 38 批次
请求成功！ 第 39 批次
请求成功！ 第 40 批次
请求成功！ 第 41 批次
请求成功！ 第 42 批次
请求成功！ 第 43 批次
请求成功！ 第 44 批次
请求成功！ 第 45 批次
请求成功！ 第 46 批次
请求成功！ 第 47 批次
请求成功！ 第 48 批次
请求成功！ 第 49 批次
请求成功！ 第 50 批次
请求成功！ 第 51 批次
请求成功！ 第 52 批次
请求成功！ 第 53 批次
请求成功！ 第 54 批次
请求成功！ 第 55 批次
请求成功！ 第 56 批次
请求成功！ 第 57 批次
请求成功！ 第 58 批次
请求成功！ 第 59 批次
请求成功！ 第 60 批次
请求成功！ 第 61 批次
请求成功！ 第 62 批次
请求成功！ 第 63 批次
请求成功！ 第 64 批次
请求成功！ 第 65 批次
请求成功！ 第 66 批次
请求成功！ 第 67 批次
请求成功！ 第 68 批次
请求成功！ 第 69 批次
请求成功！ 第 70 批次
请求成功！ 第 71 批次
请求成功！ 第 72 批次
请

In [51]:
gene2ko.shape

(9256, 2)

In [53]:
gene2ko.head()

,org:gene,ko
0,shh:ShL2_00101,ko:K00852
1,pfp:PFL1_04946,ko:K02980
2,pfp:PFL1_04823,ko:K03009
3,pfp:PFL1_03718,ko:K02147
4,pfp:PFL1_02877,ko:K02976


In [52]:
# 如果你想将结果保存为文件，可以使用以下代码：
with open('kegg_results.txt', 'w') as f:
    f.write(final_result)

再看ko在hsa上又能对应多少基因

In [61]:
import requests
import time

def webtext2df_ko2gene(x):
    import pandas as pd
    rows = x.split('\n')
    data = [row.split('\t') for row in rows]
    df = pd.DataFrame(data, columns=['ko','hsa:gene'])
    df.drop(df.index[-1],inplace=True)
    return df

# 将 keggid_list 按 100 行分块
chunk_size = 100

# 初始化存储所有请求结果的变量
all_results = []

# 循环每个分块进行请求
for i in range(0, len(gene2ko['ko']), chunk_size):
    # 获取当前的 100 行数据
    current_chunk = gene2ko['ko'][i:i + chunk_size]
    
    # 将列表转换为以加号（+）连接的字符串，适合 URL 请求格式
    chunk_str = '+'.join(current_chunk)
    
    # 构建请求 URL
    url = f'https://rest.kegg.jp/link/hsa/{chunk_str}'
    
    try:
        # 发送 GET 请求
        response = requests.get(url)
        
        if response.status_code == 200:
            print(f"请求成功！ 第 {i//chunk_size + 1} 批次")
            # 将请求的文本结果添加到总结果中
            all_results.append(response.text)
        else:
            print(f"请求失败！ HTTP 状态码: {response.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"请求发生错误: {e}")
    
    # 每次请求后睡眠2秒
    time.sleep(2)

# 将所有结果汇总为一个字符串
final_result = ''.join(all_results)

ko2hsa = webtext2df_ko2gene(final_result)

请求成功！ 第 1 批次
请求成功！ 第 2 批次
请求成功！ 第 3 批次
请求成功！ 第 4 批次
请求成功！ 第 5 批次
请求成功！ 第 6 批次
请求成功！ 第 7 批次
请求成功！ 第 8 批次
请求成功！ 第 9 批次
请求成功！ 第 10 批次
请求成功！ 第 11 批次
请求成功！ 第 12 批次
请求成功！ 第 13 批次
请求成功！ 第 14 批次
请求成功！ 第 15 批次
请求成功！ 第 16 批次
请求成功！ 第 17 批次
请求成功！ 第 18 批次
请求成功！ 第 19 批次
请求成功！ 第 20 批次
请求成功！ 第 21 批次
请求成功！ 第 22 批次
请求成功！ 第 23 批次
请求成功！ 第 24 批次
请求成功！ 第 25 批次
请求成功！ 第 26 批次
请求成功！ 第 27 批次
请求成功！ 第 28 批次
请求成功！ 第 29 批次
请求成功！ 第 30 批次
请求成功！ 第 31 批次
请求成功！ 第 32 批次
请求成功！ 第 33 批次
请求成功！ 第 34 批次
请求成功！ 第 35 批次
请求成功！ 第 36 批次
请求成功！ 第 37 批次
请求成功！ 第 38 批次
请求成功！ 第 39 批次
请求成功！ 第 40 批次
请求成功！ 第 41 批次
请求成功！ 第 42 批次
请求成功！ 第 43 批次
请求成功！ 第 44 批次
请求成功！ 第 45 批次
请求成功！ 第 46 批次
请求成功！ 第 47 批次
请求成功！ 第 48 批次
请求成功！ 第 49 批次
请求成功！ 第 50 批次
请求成功！ 第 51 批次
请求成功！ 第 52 批次
请求成功！ 第 53 批次
请求成功！ 第 54 批次
请求成功！ 第 55 批次
请求成功！ 第 56 批次
请求成功！ 第 57 批次
请求成功！ 第 58 批次
请求成功！ 第 59 批次
请求成功！ 第 60 批次
请求成功！ 第 61 批次
请求成功！ 第 62 批次
请求成功！ 第 63 批次
请求成功！ 第 64 批次
请求成功！ 第 65 批次
请求成功！ 第 66 批次
请求成功！ 第 67 批次
请求成功！ 第 68 批次
请求成功！ 第 69 批次
请求成功！ 第 70 批次
请求成功！ 第 71 批次
请求成功！ 第 72 批次
请

In [62]:
ko2hsa.shape

(7421, 2)

In [63]:
ko2hsa.head(10)

,ko,hsa:gene
0,ko:K00852,hsa:64080
1,ko:K02980,hsa:6235
2,ko:K03009,hsa:5440
3,ko:K02147,hsa:525
4,ko:K02147,hsa:526
5,ko:K02976,hsa:6231
6,ko:K10573,hsa:7319
7,ko:K02954,hsa:63931
8,ko:K02874,hsa:64928
9,ko:K02919,hsa:64979


In [68]:
hsagene_list = '+'.join(ko2hsa['hsa:gene'].head(10)

# url
url = f'https://rest.kegg.jp/get/{hsagene_list}'

# 发送 GET 请求 并获取响应
response = requests.get(url)

if response.status_code == 200:
    print("请求成功！")
else :
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

请求成功！


In [80]:
def parse_kegg_response(data):
    # 分割多个基因的内容块，每个块由 "///" 分隔
    gene_blocks = data.split('///')
    
    # 初始化一个空的列表来保存解析的结果
    gene_data = []

    # 遍历每个基因块
    for block in gene_blocks:
        gene_id = None
        gene_symbol = None
        # gene_name = None
        
        # 将每个块按行分割
        lines = block.strip().split('\n')
        
        for line in lines:
            # 找到ENTRY行，提取gene_id
            if line.startswith("ENTRY"):
                gene_id = 'hsa:' + line.split()[1]  # 提取ENTRY后面的gene_id
            # 找到SYMBOL行，提取第一个基因符号
            elif line.startswith("SYMBOL"):
                gene_symbol = line.split() 
                if len(gene_symbol) > 2:
                    gene_symbol = gene_symbol[1].split(',')[0]
                else :
                    gene_symbol = gene_symbol[1] 
            # 找到NAME行，提取基因名
            # elif line.startswith("NAME"):
                # gene_name = line.split()[1]  # 提取NAME后面的基因名
        
        # 如果找到了gene_id和gene_symbol，将它们添加到结果列表中
        if gene_id and gene_symbol:
            gene_data.append([gene_id, gene_symbol])
    
    # 将结果转换为DataFrame
    df = pd.DataFrame(gene_data, columns=['Gene_ID', 'Symbol'])
    return df




In [81]:
hsagene_list = '+'.join(ko2hsa['hsa:gene'].head(10))

# url
url = f'https://rest.kegg.jp/get/{hsagene_list}'

# 发送 GET 请求 并获取响应
response = requests.get(url)

if response.status_code == 200:
    print("请求成功！")
else :
    print(f"Failed to retrieve data. HTTP Status Code: {response.status_code}")

gene2symbol = parse_kegg_response(response.text)

请求成功！


In [82]:
gene2symbol

,Gene_ID,Symbol
0,hsa:64080,RBKS
1,hsa:6235,RPS29
2,hsa:5440,POLR2K
3,hsa:525,ATP6V1B1
4,hsa:526,ATP6V1B2
5,hsa:6231,RPS26
6,hsa:7319,UBE2A
7,hsa:63931,MRPS14
8,hsa:64928,MRPL14
9,hsa:64979,MRPL36


In [83]:
import requests
import time

# parse_kegg_response 把请求得到的html提取gene entry和gene symbol
def parse_kegg_response(data):
    # 分割多个基因的内容块，每个块由 "///" 分隔
    gene_blocks = data.split('///')
    
    # 初始化一个空的列表来保存解析的结果
    gene_data = []

    # 遍历每个基因块
    for block in gene_blocks:
        gene_id = None
        gene_symbol = None
        # gene_name = None
        
        # 将每个块按行分割
        lines = block.strip().split('\n')
        
        for line in lines:
            # 找到ENTRY行，提取gene_id
            if line.startswith("ENTRY"):
                gene_id = 'hsa:' + line.split()[1]  # 提取ENTRY后面的gene_id
            # 找到SYMBOL行，提取第一个基因符号
            elif line.startswith("SYMBOL"):
                gene_symbol = line.split() 
                if len(gene_symbol) > 2:
                    gene_symbol = gene_symbol[1].split(',')[0]
                else :
                    gene_symbol = gene_symbol[1] 
            # 找到NAME行，提取基因名
            # elif line.startswith("NAME"):
                # gene_name = line.split()[1]  # 提取NAME后面的基因名
        
        # 如果找到了gene_id和gene_symbol，将它们添加到结果列表中
        if gene_id and gene_symbol:
            gene_data.append([gene_id, gene_symbol])
    
    # 将结果转换为DataFrame
    df = pd.DataFrame(gene_data, columns=['Gene_ID', 'Symbol'])
    return df


# 将 keggid_list 按 100 行分块
chunk_size = 100

# 初始化存储所有请求结果的变量
all_results = []

# 循环每个分块进行请求
for i in range(0, len(ko2hsa['hsa:gene']), chunk_size):
    # 获取当前的 100 行数据
    current_chunk = ko2hsa['hsa:gene'][i:i + chunk_size]
    
    # 将列表转换为以加号（+）连接的字符串，适合 URL 请求格式
    chunk_str = '+'.join(current_chunk)
    
    # 构建请求 URL
    url = f'https://rest.kegg.jp/get/{chunk_str}'
    
    try:
        # 发送 GET 请求
        response = requests.get(url)
        
        if response.status_code == 200:
            print(f"请求成功！ 第 {i//chunk_size + 1} 批次")
            # 将请求的文本结果添加到总结果中
            all_results.append(response.text)
        else:
            print(f"请求失败！ HTTP 状态码: {response.status_code}")
        
    except requests.exceptions.RequestException as e:
        print(f"请求发生错误: {e}")
    
    # 每次请求后睡眠2秒
    time.sleep(2)

# 将所有结果汇总为一个字符串
final_result = ''.join(all_results)

gene2symbol = parse_kegg_response(final_result)

请求成功！ 第 1 批次
请求成功！ 第 2 批次
请求成功！ 第 3 批次
请求成功！ 第 4 批次
请求成功！ 第 5 批次
请求成功！ 第 6 批次
请求成功！ 第 7 批次
请求成功！ 第 8 批次
请求成功！ 第 9 批次
请求成功！ 第 10 批次
请求成功！ 第 11 批次
请求成功！ 第 12 批次
请求成功！ 第 13 批次
请求成功！ 第 14 批次
请求成功！ 第 15 批次
请求成功！ 第 16 批次
请求成功！ 第 17 批次
请求成功！ 第 18 批次
请求成功！ 第 19 批次
请求成功！ 第 20 批次
请求成功！ 第 21 批次
请求成功！ 第 22 批次
请求成功！ 第 23 批次
请求成功！ 第 24 批次
请求成功！ 第 25 批次
请求成功！ 第 26 批次
请求成功！ 第 27 批次
请求成功！ 第 28 批次
请求成功！ 第 29 批次
请求成功！ 第 30 批次
请求成功！ 第 31 批次
请求成功！ 第 32 批次
请求成功！ 第 33 批次
请求成功！ 第 34 批次
请求成功！ 第 35 批次
请求成功！ 第 36 批次
请求成功！ 第 37 批次
请求成功！ 第 38 批次
请求成功！ 第 39 批次
请求成功！ 第 40 批次
请求成功！ 第 41 批次
请求成功！ 第 42 批次
请求成功！ 第 43 批次
请求成功！ 第 44 批次
请求成功！ 第 45 批次
请求成功！ 第 46 批次
请求成功！ 第 47 批次
请求成功！ 第 48 批次
请求成功！ 第 49 批次
请求成功！ 第 50 批次
请求成功！ 第 51 批次
请求成功！ 第 52 批次
请求成功！ 第 53 批次
请求成功！ 第 54 批次
请求成功！ 第 55 批次
请求成功！ 第 56 批次
请求成功！ 第 57 批次
请求成功！ 第 58 批次
请求成功！ 第 59 批次
请求成功！ 第 60 批次
请求成功！ 第 61 批次
请求成功！ 第 62 批次
请求成功！ 第 63 批次
请求成功！ 第 64 批次
请求成功！ 第 65 批次
请求成功！ 第 66 批次
请求成功！ 第 67 批次
请求成功！ 第 68 批次
请求成功！ 第 69 批次
请求成功！ 第 70 批次
请求成功！ 第 71 批次
请求成功！ 第 72 批次
请

In [84]:
gene2symbol.shape

(741, 2)

In [85]:
gene2symbol.head(10)

,Gene_ID,Symbol
0,hsa:64080,RBKS
1,hsa:6235,RPS29
2,hsa:5440,POLR2K
3,hsa:525,ATP6V1B1
4,hsa:526,ATP6V1B2
5,hsa:6231,RPS26
6,hsa:7319,UBE2A
7,hsa:63931,MRPS14
8,hsa:64928,MRPL14
9,hsa:64979,MRPL36


In [97]:
print(keggid.shape)
print(gene2ko.shape)
print(ko2hsa.shape)
print(gene2symbol.shape)

(14637, 2)
(9256, 2)
(7421, 2)
(741, 2)


In [98]:
# 如果你想将结果保存为文件，可以使用以下代码：
keggid.to_csv("./keggid_14637.csv", index=False)

gene2ko.to_csv("./gene2ko_9256.csv", index=False)

ko2hsa.to_csv("./ko2hsa_7421.csv", index=False)

gene2symbol.to_csv("./gene2symbol_741.csv", index=False)

In [108]:
ko2hsa[ko2hsa['hsa:gene'].duplicated()].head(10)

,ko,hsa:gene
38,ko:K02954,hsa:63931
69,ko:K02963,hsa:51023
70,ko:K02963,hsa:55168
79,ko:K00134,hsa:2597
129,ko:K02954,hsa:63931
130,ko:K01755,hsa:435
131,ko:K02520,hsa:219402
140,ko:K01358,hsa:8192
142,ko:K04393,hsa:998
145,ko:K11253,hsa:440093


In [106]:
gene2symbol[gene2symbol['Symbol'].duplicated()].head(10)

,Gene_ID,Symbol
59,hsa:64979,MRPL36
60,hsa:8350,H3C1
61,hsa:3020,H3-3A
62,hsa:8290,H3-4
63,hsa:126961,H3C14
64,hsa:333932,H3C15
65,hsa:653604,H3C13
70,hsa:8351,H3C4
71,hsa:8352,H3C3
72,hsa:8350,H3C1


In [105]:
import numpy as np
np.where(gene2symbol['Symbol']=='H3C1')

(array([ 44,  60,  72,  84, 445, 533, 539]),)

In [109]:
np.where(ko2hsa['hsa:gene']=='hsa:63931')

(array([   7,   38,  129,  328,  486,  946,  988, 1016, 1025, 1069, 4144,
        4823, 5318, 6014, 6233, 6240, 6312, 6514, 6591, 6647, 6712, 6726,
        6766, 6909]),)

In [110]:
np.where(gene2ko['ko']=='ko:K02954')

(array([   6,   15,  247,  714,  983, 1370, 1478, 1513, 1532, 1599, 4585,
        5715, 6504, 7048, 7342, 7352, 7480, 7734, 7870, 8033, 8217, 8242,
        8297, 8586]),)

In [119]:
gene2ko.iloc[6]

org:gene    asg:FB03_03870
ko               ko:K02954
Name: 6, dtype: object

In [120]:
gene2ko.iloc[15]

org:gene    amq:AMETH_0665
ko               ko:K02954
Name: 15, dtype: object

In [121]:
ko2hsa[ko2hsa['ko'].duplicated()].head(10)

,ko,hsa:gene
4,ko:K02147,hsa:526
12,ko:K05692,hsa:60
14,ko:K11254,hsa:554313
15,ko:K11254,hsa:8294
16,ko:K11254,hsa:8360
17,ko:K11254,hsa:8361
18,ko:K11254,hsa:8362
19,ko:K11254,hsa:8363
20,ko:K11254,hsa:8364
21,ko:K11254,hsa:8365


In [122]:
ko2hsa[ko2hsa['hsa:gene'].duplicated()].head(10)

,ko,hsa:gene
38,ko:K02954,hsa:63931
69,ko:K02963,hsa:51023
70,ko:K02963,hsa:55168
79,ko:K00134,hsa:2597
129,ko:K02954,hsa:63931
130,ko:K01755,hsa:435
131,ko:K02520,hsa:219402
140,ko:K01358,hsa:8192
142,ko:K04393,hsa:998
145,ko:K11253,hsa:440093


In [123]:
gene2ko[gene2ko['ko'].duplicated()].head(10)

,org:gene,ko
15,amq:AMETH_0665,ko:K02954
88,cuv:CUREI_02510,ko:K18955
99,cuv:CUREI_03700,ko:K02963
112,cuv:CUREI_06445,ko:K00134
142,cuv:CUREI_08750,ko:K07052
167,cuv:CUREI_10130,ko:K03686
182,cuv:CUREI_00715,ko:K18955
198,cuv:CUREI_05305,ko:K15977
225,sya:A6768_24555,ko:K08998
227,hml:HmaOT1_18910,ko:K07497


In [124]:
gene2ko[gene2ko['org:gene'].duplicated()].head(10)

,org:gene,ko
365,sep:SE_1541,ko:K23059
4989,ckp:ckrop_0092,ko:K10823
5372,ckp:ckrop_1085,ko:K02034
5533,ckp:ckrop_1506,ko:K00652
6293,pak:HMPREF0675_4919,ko:K00652
6502,pon:100439869,ko:K07374
6860,zma:845193,ko:K02637
6924,hsa:5175,ko:K06471
7065,hsa:10376,ko:K07374
9169,hsa:100288687,ko:K24896


In [125]:
keggid.head(50)

,From,KEGG
0,A0A023UEV9,shh:ShL2_00055
1,A0A023UGA4,shh:ShL2_00101
2,A0A028ZLY2,shh:ShL2_00073
3,A0A031FNR5,moo:BWL13_02343
4,A0A060AB44,vg:19685786
5,A0A060AB89,vg:19685835
6,A0A060ABD0,vg:19685880
7,A0A060AET7,vg:19685756
8,A0A060AF10,vg:19685841
9,A0A060AF31,vg:19685866
